# Segementing and clustering neighbourhoods in Toronto, Canada

## Part 1 - Web scraping into a Dataframe

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 



Solving environment: done

# All requested packages already installed.



In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
Toronto_df = df[0]
print(Toronto_df.head(5))
Toronto_df.shape

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


(287, 3)

In [3]:
#Remove data with no Borough data
TO_ALL_df = Toronto_df[Toronto_df.Borough != "Not assigned"]
#Neighbourhoods with no name assigned Borough name
TO_ALL_df.loc[TO_ALL_df['Neighbourhood'] == "Not assigned", 'Borough'] = 1
#Column Postcode renamed to Postalcode
TO_ALL_df.rename(columns = {"Postcode": 'Postalcode'}, inplace = True)
#Combine neighbourhoods with the same postalcode into one column separated by a comma 
TO_df = TO_ALL_df.groupby(['Postalcode','Borough'], sort = 'Postalcode').agg(lambda x: ', '.join(x))
TO_df = TO_df.reset_index()
TO_df



/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [4]:
TO_df.shape

(103, 3)

## Part 2 - Adding location data using .csv file

In [5]:
coo_df = pd.read_csv('http://cocl.us/Geospatial_data', )
coo_df.rename({'Postal Code': 'Postalcode'}, axis=1, inplace=True)
#display(coo_df)
TO_df = TO_df.merge(coo_df.set_index('Postalcode'), on='Postalcode')
TO_df

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Part 3 - Exploring and clustering 

In [6]:
print(TO_df.groupby(['Borough']).count())


# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[TO_df["Latitude"].mean(), TO_df['Longitude'].mean()], zoom_start=11)

# add markers to map
for lat, lng, label in zip(TO_df['Latitude'], TO_df['Longitude'], TO_df['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


                  Postalcode  Neighbourhood  Latitude  Longitude
Borough                                                         
Central Toronto            9              9         9          9
Downtown Toronto          19             19        19         19
East Toronto               5              5         5          5
East York                  5              5         5          5
Etobicoke                 12             12        12         12
Mississauga                1              1         1          1
North York                24             24        24         24
Scarborough               17             17        17         17
West Toronto               6              6         6          6
York                       5              5         5          5


### Only neighbourhoods from Downtown Toronto was explored further 

In [7]:
downtown = TO_ALL_df[TO_ALL_df['Borough'] == 'Downtown Toronto']
#coo_df.rename({'Postal Code': 'Postalcode'}, axis=1, inplace=True)
#coo_df.head(5)
downtown = downtown.join(coo_df.set_index('Postalcode'), on='Postalcode')
print(downtown.shape)
downtown.head(10)

(37, 5)


,Postalcode,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
7,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
16,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
17,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
33,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
47,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
56,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
57,M6G,Downtown Toronto,Christie,43.669542,-79.422564
67,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568
68,M5H,Downtown Toronto,King,43.650571,-79.384568


### Explore using Foursquare

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
# @hidden_cell


In [10]:

LIMIT = 100
 

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)


downtown_venues = getNearbyVenues(names=downtown['Neighbourhood'],
                                   latitudes=downtown['Latitude'],
                                   longitudes=downtown['Longitude']
                                  )
downtown_venues

Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
5,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,Harbourfront,43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
7,Harbourfront,43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
8,Harbourfront,43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
9,Harbourfront,43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [11]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped.head()



,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,...,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,Bathurst Quay,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,CN Tower,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,Cabbagetown,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [12]:
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0       Coffee Shop  0.07
1        Restaurant  0.05
2              Café  0.04
3   Thai Restaurant  0.04
4  Sushi Restaurant  0.03


----Bathurst Quay----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4           Airport  0.06


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1              Bakery  0.04
2                Café  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04


----CN Tower----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4           Airport  0.06


----Cabbagetown----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.07
2          Restaurant  0.07
3  Italian Restaurant  0.04
4              Bakery  0.04


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.16
1   I

In [13]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Sushi Restaurant,Bookstore,Asian Restaurant,Pizza Place,Burger Joint
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court
2,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Café,Restaurant,Cheese Shop,Eastern European Restaurant
3,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court
4,Cabbagetown,Café,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Pizza Place,Pub,Indian Restaurant,Japanese Restaurant,Sandwich Place


### Six clusters created 

In [14]:
# set number of clusters
kclusters = 6

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.merge(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='left')

downtown_merged.head() # check the last columns!



[2 1 2 1 2 2 3 5 2 2]


,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2,Coffee Shop,Park,Café,Pub,Bakery,Mexican Restaurant,Breakfast Spot,Restaurant,Theater,Electronics Store
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,2,Coffee Shop,Park,Burger Joint,Beer Bar,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,Chinese Restaurant,College Auditorium
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,2,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Theater,Lingerie Store,Ramen Restaurant
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,2,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Theater,Lingerie Store,Ramen Restaurant
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Diner,Breakfast Spot,Hotel,Japanese Restaurant,Clothing Store


In [15]:

for cluster in range(6): 
    display(downtown_merged.loc[downtown_merged['Cluster Labels'] == cluster, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]])
    downtown_clusters = downtown_merged.loc[downtown_merged['Cluster Labels'] == cluster, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]
    print('\n')


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Downtown Toronto,0,Café,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Restaurant,Sandwich Place,Beer Bar,Beer Store
19,Downtown Toronto,0,Café,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Restaurant,Sandwich Place,Beer Bar,Beer Store


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court
24,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court
25,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court
26,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court
27,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court
28,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court
29,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Park,Café,Pub,Bakery,Mexican Restaurant,Breakfast Spot,Restaurant,Theater,Electronics Store
1,Downtown Toronto,2,Coffee Shop,Park,Burger Joint,Beer Bar,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,Chinese Restaurant,College Auditorium
2,Downtown Toronto,2,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Theater,Lingerie Store,Ramen Restaurant
3,Downtown Toronto,2,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Theater,Lingerie Store,Ramen Restaurant
4,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Diner,Breakfast Spot,Hotel,Japanese Restaurant,Clothing Store
5,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Café,Restaurant,Cheese Shop,Eastern European Restaurant
6,Downtown Toronto,2,Coffee Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Ice Cream Shop,Burger Joint,Sandwich Place,Thai Restaurant,Department Store,Sushi Restaurant
8,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Sushi Restaurant,Bookstore,Asian Restaurant,Pizza Place,Burger Joint
9,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Sushi Restaurant,Bookstore,Asian Restaurant,Pizza Place,Burger Joint
10,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Sushi Restaurant,Bookstore,Asian Restaurant,Pizza Place,Burger Joint


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,3,Bar,Vietnamese Restaurant,Bakery,Café,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Comfort Food Restaurant
21,Downtown Toronto,3,Bar,Vietnamese Restaurant,Bakery,Café,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Comfort Food Restaurant
22,Downtown Toronto,3,Bar,Vietnamese Restaurant,Bakery,Café,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Comfort Food Restaurant


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Downtown Toronto,4,Park,Playground,Trail,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,5,Grocery Store,Café,Park,Coffee Shop,Gas Station,Diner,Italian Restaurant,Restaurant,Baby Store,Candy Store


In [16]:

# create map of Toronto using latitude and longitude values
map_downtown = folium.Map(location=[downtown_merged["Latitude"].mean(), downtown_merged['Longitude'].mean()], zoom_start=13)
color_select = ['red','blue','gray','purple','green','orange']
# add markers to map
for lat, lng, label, col in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['1st Most Common Venue'],downtown_merged['Cluster Labels']):
    label = folium.Popup(label, parse_html=True)
    #col = int(col)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color_select[col],
        fill=True,
        fill_color= color_select[col],
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)     
map_downtown